## Import

In [10]:
from __future__ import print_function, division

import os
import numpy as np
import fitsio

import fsps
import sedpy
import prospect

from sedpy.observate import load_filters

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Setup the models

In [3]:
from prospect.models import model_setup
from prospect.sources import CSPBasis

In [5]:
#help(CSPBasis)

In [6]:
# Specify the run parameters.
run_params = {
    'param_file': '',
    'verbose': True,
    'debug': False,
              'outfile': 'redmapper-masses',
              # Fitter parameters
              'nwalkers': 128,
              'nburn': [32, 32, 64], 'niter': 512,
              'do_powell': True,
              'ftol': 0.5e-5, 'maxfev': 5000,
              'initial_disp': 0.1,
              # Data parameters
              #'snr': 20.0, # ???
              #'add_noise': False, # ???
              # Model parameters
              'mass': 1E10,
              'logzsol': -0.5,
              'tage': 12.0,
              'tau': 3.0,
              'dust2': 0.3,
              # Data manipulation parameters
              #'logify_spectrum': False,
              #'normalize_spectrum': False,
              #'wlo':3750., 'whi':7200.,
              # SPS parameters
              'zcontinuous': 1,
              }
print(run_params)

{'niter': 512, 'dust2': 0.3, 'verbose': True, 'tage': 12.0, 'logzsol': -0.5, 'do_powell': True, 'zcontinuous': 1, 'initial_disp': 0.1, 'tau': 3.0, 'outfile': 'redmapper-masses', 'nburn': [32, 32, 64], 'maxfev': 5000, 'debug': False, 'mass': 10000000000.0, 'nwalkers': 128, 'param_file': '', 'ftol': 5e-06}


In [7]:
# load sps model (default)
#sps = model_setup.load_sps(**run_params)
sps = CSPBasis(zcontinuous=1, compute_vega_mags=False)

In [8]:
sdss = ['sdss_{}0'.format(b) for b in ['u','g','r','i','z']]
wise = ['wise_{}'.format(b) for b in ['w1','w2']]
filternames = sdss + wise
print(filternames)

['sdss_u0', 'sdss_g0', 'sdss_r0', 'sdss_i0', 'sdss_z0', 'wise_w1', 'wise_w2']


In [11]:
filters = load_filters(filternames)
print(filters)
#help(filters[0])
#filters[0].load_filter
#filters[0].display()

[<class 'sedpy.observate.Filter'>(sdss_u0), <class 'sedpy.observate.Filter'>(sdss_g0), <class 'sedpy.observate.Filter'>(sdss_r0), <class 'sedpy.observate.Filter'>(sdss_i0), <class 'sedpy.observate.Filter'>(sdss_z0), <class 'sedpy.observate.Filter'>(wise_w1), <class 'sedpy.observate.Filter'>(wise_w2)]


In [12]:
# create a function to read in the redmapper catalog and extract relavent files

def readred():
    
    redfile = os.path.join(os.sep, 'global', 'work', 'projects', 'redmapper', 'redmapper_isedfit_v5.10_centrals.fits.gz')
    info = fitsio.FITS(redfile)
    cat = info[1].read()
    
    return cat

#redfile

In [13]:
# Build output dictionary. 


def prospcat(redcat):
    
    mask = (redcat['IVARMAGGIES'] > 0) * 1
    # print(mask)
    
    obs = {}
    obs['maggies'] = redcat['MAGGIES']
    with np.errstate(divide='ignore'):
        obs['maggies_unc'] = 1.0/np.sqrt(redcat['IVARMAGGIES']) #[:3, :])
    obs['wavelength'] = None # not fitting spectra
    obs['filters'] = filters
    obs['phot_mask'] = mask  # 1 = good, 0 = bad
    obs['isedfit_id'] = redcat['ISEDFIT_ID']
    
    return obs
    

In [14]:
redcat = readred()

In [16]:
#redcat.dtype.names

In [17]:
obs = prospcat(redcat)

In [1]:
#print(obs.keys())
#print()
#print(-2.5*np.log10(obs['maggies']))
#print()
#print(2.5*obs['maggies_unc']/(np.log(10)*obs['maggies']))